In [1]:
import csv
import re
import re
import csv
import nltk


In [3]:
#start process_tweet
def processTweet(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end




In [4]:
#initialize stopWords
stopWords = []

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end



In [5]:
#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end


In [6]:

#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
#end

In [7]:
#Read the tweets one by one and process it
inpTweets = csv.reader(open('C:\Users\hp\Documents\Projects5\classifiers\labeled_Dataset.csv', 'rb'), delimiter=',', quotechar='|')
tweets = []
feature = []
featureList = []
stopWords = getStopWordList('C:/Users/hp/Documents/Projects5/classifiers/stopwords.txt')

for row in inpTweets:
    sentiment = row[0]
    tweet = row[1]
    processedTweet = processTweet(tweet)
    featureVector = getFeatureVector(processedTweet)
    feature.append(featureVector)
    tweets.append((featureVector, sentiment));
#end loop

# save all the unigrams features in a list
for li in feature:
    for l in li:
        featureList.append(l)

#end saving


In [8]:



#start extract_features
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features
#end




In [9]:
     
# Remove featureList duplicates
featureList = list(set(featureList))



In [10]:
# Extract feature vector for all tweets in one shote
training_set = nltk.classify.util.apply_features(extract_features, tweets)



In [11]:
# Train the classifier -----NAIVE BAYES---
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)



In [12]:
# Test the classifier
testTweet = 'nah soo booring '
processedTestTweet = processTweet(testTweet)
print NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))
print NBClassifier.show_most_informative_features(10)

0
Most Informative Features
           contains(sad) = True                0 : 1      =     73.0 : 1.0
   contains(musicmonday) = True                1 : 0      =     38.8 : 1.0
  contains(followfriday) = True                1 : 0      =     19.3 : 1.0
       contains(missing) = True                0 : 1      =     14.6 : 1.0
       contains(anymore) = True                0 : 1      =     14.3 : 1.0
         contains(didnt) = True                0 : 1      =     13.0 : 1.0
         contains(upset) = True                0 : 1      =     13.0 : 1.0
         contains(hates) = True                0 : 1      =     12.3 : 1.0
      contains(horrible) = True                0 : 1      =     11.7 : 1.0
          contains(sexy) = True                1 : 0      =     11.7 : 1.0
None


In [13]:
from sklearn.externals import joblib
joblib.dump(processTweet, 'senti_classifier.pkl')



In [ ]:
#load the NAIVE BAYES classifier we already build
classifier = joblib.load('senti_classifier.pkl')

In [25]:
# Test the classifier
testTweet = 'actually i coundnt imagine he was that baaaaad '
processedTestTweet = classifier(testTweet)
print NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))
print NBClassifier.show_most_informative_features(10)

0
Most Informative Features
           contains(sad) = True                0 : 1      =     73.0 : 1.0
   contains(musicmonday) = True                1 : 0      =     38.8 : 1.0
  contains(followfriday) = True                1 : 0      =     19.3 : 1.0
       contains(missing) = True                0 : 1      =     14.6 : 1.0
       contains(anymore) = True                0 : 1      =     14.3 : 1.0
         contains(didnt) = True                0 : 1      =     13.0 : 1.0
         contains(upset) = True                0 : 1      =     13.0 : 1.0
         contains(hates) = True                0 : 1      =     12.3 : 1.0
      contains(horrible) = True                0 : 1      =     11.7 : 1.0
          contains(sexy) = True                1 : 0      =     11.7 : 1.0
None
